In [1]:
# In this script, we will plot the adjusted 4xCO2 at the TOA from WACCM PORT simulations

# By: Ty Janoski
# Updated: 06.08.20

In [17]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
import xesmf as xe
import numpy as np
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib
from mpl_toolkits.axes_grid1 import AxesGrid
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [10]:
# we also need the adjusted TOA radiative perturbation from 4xCO2
nums = [str(n).zfill(2) for n in np.arange(1,13,1)]
diff = xr.concat([xr.open_dataset('/dx01/chiodo/data2/waccm/f1850.wcm.port.4xCO2.adj.001/diff.'+n+'.RAD.nc',
                                decode_times=False,drop_variables=['TCOOR','hyam','hybm','lev']) for n in nums],dim='time')
diff['time'] = np.arange(1,13,1)
rad=diff

In [11]:
ds_out = xr.Dataset({'lat': (['lat'], rad.lat),
                     'lon': (['lon'], np.arange(0,360.1,2.5))})
regridder = xe.Regridder(rad, ds_out, 'bilinear',reuse_weights=True, periodic=True)
FLNT = regridder(rad['FLNT'])
FSNT = regridder(rad['FSNT'])
FLNR = regridder(rad['FLNR'])
FSNR = regridder(rad['FSNR'])

Create weight file: bilinear_96x144_96x145_peri.nc


In [14]:
ann_FLNT = -1 *FLNT.mean(dim='time')
ann_FSNT = FSNT.mean(dim='time')
net = ann_FLNT + ann_FSNT

ann_FLNR = -1 *FLNR.mean(dim='time')
ann_FSNR = FSNR.mean(dim='time')
net_R = ann_FLNR + ann_FSNR

net_R_m1 = (-1 * FLNR).isel(time=0) + FSNR.isel(time=0)
net_m1 = (-1 * FLNT).isel(time=0) + FSNT.isel(time=0)

In [15]:
def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero.

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower offset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to 
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax / (vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highest point in the colormap's range.
          Defaults to 1.0 (no upper offset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }
    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)

    return newcmap

In [28]:
fig = plt.figure(figsize=(6,3.5))
proj = ccrs.Robinson()
axes_class = (GeoAxes,
                  dict(map_projection=proj))

axgr = AxesGrid(fig, 111, axes_class=axes_class,
                nrows_ncols=(1, 1),
                axes_pad=0.6,
                cbar_location='right',
                cbar_mode='single',
                cbar_pad=0.05,
                cbar_size='3%',
                label_mode='')
axgr[0].coastlines(resolution='110m')
cf = axgr[0].contourf(net.lon,net.lat,net,transform=ccrs.PlateCarree(),levels=np.arange(0,12,1),cmap=cm.Reds)
cb = axgr.cbar_axes[0].colorbar(cf,ticks=np.arange(0,11,2))
axgr.cbar_axes[0].axis[axgr.cbar_axes[0].orientation].label.set_text(r'Annual mean rad. forc. (Wm$^{-2}$)')
axgr.cbar_axes[0].axis[axgr.cbar_axes[0].orientation].label.set_size(7)
cb.ax.tick_params(labelsize=6)

# axgr[0].set_title('Annual mean radiative forcing',fontsize=8)

# plt.show()
plt.savefig('Fig_S1_060820.pdf')

<Figure size 432x252 with 2 Axes>